coffee\_disease\_detection.py


In [3]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

2025-12-04 16:03:23.943060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 16:03:23.979982: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 16:03:25.257810: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 16:03:25.835041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764864206.269294     368 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764864206.34

1. Reproducibility settings



In [0]:
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

2. Dataset Parameters



In [0]:
IMG_SIZE = (256, 256)
BATCH_SIZE = 100
DATA_DIR = 'dataset'  

# Root directory containing train/val/test subfolders
# data_dir = pathlib.Path)('my_image_data')
# (data_dir /'Train').mkdir(parents=True, exist_ok=True)
# (data_dir /'Validation').mkdir(parents=True, exist_ok=True)
# (data_dir /'Test').mkdir(parents=True, exist_ok=True)


3. Data Loading



In [0]:
def get_datasets(data_dir, img_size, batch_size, seed):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(data_dir, 'train'),
        labels='inferred',
        label_mode='categorical',
        batch_size=batch_size,
        image_size=img_size,
        shuffle=True,
        seed=seed
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(data_dir, 'val'),
        labels='inferred',
        label_mode='categorical',
        batch_size=batch_size,
        image_size=img_size,
        shuffle=False,
        seed=seed
    )
    test_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(data_dir, 'test'),
        labels='inferred',
        label_mode='categorical',
        batch_size=batch_size,
        image_size=img_size,
        shuffle=False,
        seed=seed
    )
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_datasets(DATA_DIR, IMG_SIZE, BATCH_SIZE, SEED)
class_names = train_ds.class_names